<a href="https://colab.research.google.com/github/PurabPatel555/Pytorch-Diabetic-Retinopathy/blob/master/drapp_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Set path to save trained model to 
PATH = '/content/drive/My Drive/drapp/trained_model.pt'

In [ ]:
#Installs
!pip install pretrainedmodels
!pip install tqdm

#Imports
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as f
import torch.optim as optim
from torch.utils import data
from torchvision import datasets, transforms, models
from tqdm import tqdm
from torch.utils.data.sampler import SubsetRandomSampler

#Some Settings and Hyperparameters
test_split = 0.1
shuffle_dataset = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
bs = 32
test_bs = 16
learning_rate = 0.001

#Datasets (The images are sorted into 5 folders as per their category)
root_dir = '/content/drive/My Drive/drapp/gaussian_filtered_images'
dataset = datasets.ImageFolder(root = root_dir, 
                transform = transforms.ToTensor())
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(test_split*dataset_size))
if shuffle_dataset:
  np.random.seed(42)
  np.random.shuffle(indices)

train_idx, test_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
test_sampler = SubsetRandomSampler(test_idx)

print(dataset.class_to_idx) #Check the labelling

#Dataloaders
loader = torch.utils.data.DataLoader(dataset, batch_size=bs, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(dataset, batch_size=test_bs, sampler=test_sampler)

#Create Model
class VGG16(nn.Module):
  def __init__(self):
    super(VGG16, self).__init__()
    vgg16 = models.vgg16_bn(pretrained=True)
    for param in vgg16.parameters():
      param.requires_grad = False
    num_features = vgg16.classifier[6].in_features
    features = list(vgg16.classifier.children())[:-1]
    features.extend([nn.Linear(num_features, 5)])
    vgg16.classifier = nn.Sequential(*features)
    self.model = vgg16
    
  def forward(self, image):
    bs, _, _, _ = image.shape
    out = self.model(image)
    return out

#Initialize Model
model = VGG16().to(device)
model.load_state_dict(torch.load(PATH, map_location=device))
model.train()

#Loss, Optimizer, Scheduler (Scheduler Not Used Here)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Training
def train(model, criterion, optimizer, num_epochs=epochs):
  for epoch in range(num_epochs):
    loop = tqdm(enumerate(loader), total=len(loader), leave=False)
    for batch_id, (data, targets) in loop:
      data = data.to(device=device)
      targets = targets.to(device=device)

      scores = model(data)
      loss = criterion(scores, targets)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

      loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
      loop.set_postfix(loss = loss.item())
    torch.save(model.state_dict(), PATH)
  return model

In [ ]:
#-----Train and Save-----
trained_model = train(model, criterion, optimizer, num_epochs=25)
torch.save(trained_model.state_dict(), PATH)

In [ ]:
#-----Evaluation-----
trained_model = VGG16()
trained_model.load_state_dict(torch.load(PATH, map_location=device))
trained_model.eval()

running_accuracy = 0
for _,(x, y) in enumerate(test_loader):
    x = x.to(device)
    y = y.to(device)
    _,predictions = torch.max(trained_model(x),1)
    groundtruths = y

    print(predictions)
    print(groundtruths)
    accuracy = predictions.eq(groundtruths).sum().item()
    accuracy = accuracy*100/16
    print(accuracy)
    running_accuracy += accuracy 
print(running_accuracy/len(test_loader))